In [4]:
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Software, Process, File
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from Block_Families.General._library.convert_n_and_e import convert_node
import json
import os
from Utilities.util import conv
results_base = "../Orchestration/Results/examples/"
unattached_name = "unattached.json"
attached_name = "attached.json"
promoted_name = "promoted.json"
p_nodes = []
p_edges = []
a_nodes = []
a_edges = []
u_nodes = []
u_edges = []
#
# UNATTACHED STUFF
# A. Software
software = Software(name="evil.exe")
nodes, edges = convert_node(conv(software))
nodes[0]["category"] = "unattached"
u_nodes = u_nodes + nodes
u_edges = u_edges + edges
#
# ATTACHED STUFF
# B. File
hashes ={"SHA-256": "fe90a7e910cb3a4739bed9180e807e93fa70c90f25a8915476f5e4bfbac681db"}
file = File(name="evil.exe", hashes=hashes)
nodes2, edges2 = convert_node(conv(file))
nodes2[0]["category"] = "attached"
a_nodes = a_nodes + nodes2
a_edges = a_edges + edges2
# C. Process
process = Process(pid=1221, created_time="2023-01-20T14:11:25.55Z",
                  command_line="./gedit-bin --destroy-alll", image_ref=file.id)
nodes2, edges2 = convert_node(conv(process))
nodes2[0]["category"] = "attached"
a_nodes = a_nodes + nodes2
a_edges = a_edges + edges2

sightings_file = "sightingIndex.json"
TR_Context_Memory_Dir = "./Context_Mem"
edge_name = "/incident_1/incident_edges.json"
reln_edge_name = "/incident_1/relation_edges.json"
edges =[]
reln_edges = []
with open(TR_Context_Memory_Dir + edge_name, "r") as infile:
    edges = json.load( infile)
with open(TR_Context_Memory_Dir + reln_edge_name, "r") as infile:
    reln_edges = json.load( infile)

print(f"about to open sightings")
print(f"path->",results_base + sightings_file)
if os.path.exists(results_base + sightings_file):
    sighting_children = []
    observed_children = []
    # 1. Collect all of the nodes
    with open(results_base + sightings_file, "r") as mem_input:
        todal_edges = edges + reln_edges
        sighting_tree_data = json.load(mem_input)
        print(f'total edges->', todal_edges)
        print(f'sighting_tree_data->', sighting_tree_data)
        children = sighting_tree_data["children"]
        sighting = children[0]
        sighting_children = sighting["children"]
        sighting["children"] = []
        p_nodes.append(sighting)
        for child in sighting_children:
            if child["type"] == "observed-data":
                observed_children = child["children"]
                child["children"] = []
                p_nodes.append(child)
            else:
                child["children"] = []
                p_nodes.append(child)
        for child in observed_children:
            child["children"] = []
            p_nodes.append(child)
        p_ids = [x["id"] for x in p_nodes]
        pids = {x['id']:i for i, x in enumerate(p_nodes)}
        p_edges = [x for x in todal_edges if x['source'] in p_ids and x['target'] in p_ids]
        for p in p_nodes:
            stix_index = p['id']
            p['stix_id'] = stix_index
            p['id'] =  pids[stix_index]
        for p in p_edges:
            source_val = p['source']
            source_index = pids[source_val]
            target_val = p['target']
            target_index = pids[target_val]
            p['stix_source'] = source_val
            p['stix_target'] = target_val
            p['source'] = source_index
            p['target'] = target_index

for p in p_nodes:
    p["category"] = "promoted"
for a in a_nodes:
    a["category"] = "attached"
for u in u_nodes:
    u["category"] = "unattached"

final_promoted = {"nodes": p_nodes, "edges": p_edges}
final_attached = {"nodes": a_nodes, "edges": a_edges}
final_unattached = {"nodes": u_nodes, "edges": u_edges}
print(type(final_promoted))
print(final_promoted)
print(type(final_attached))
print(final_attached)
print(type(final_unattached))
print(final_unattached)
with open(results_base + promoted_name, "w") as outfile:
    json.dump(final_promoted, outfile)
with open(results_base + attached_name, "w") as outfile:
    json.dump(final_attached, outfile)
with open(results_base + unattached_name, "w") as outfile:
    json.dump(final_unattached, outfile)


about to open sightings
path-> ../Orchestration/Results/examples/sightingIndex.json
total edges-> [{'name': 'belongs-to', 'type': 'embedded', 'source': 'email-addr--4722424c-7012-56b0-84d5-01d076fc547b', 'target': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b'}, {'name': 'email address', 'type': 'embedded', 'source': 'identity--8d87b1fd-d2d6-4c79-84f9-628d395063ea', 'target': 'email-addr--4722424c-7012-56b0-84d5-01d076fc547b'}, {'name': 'user account', 'type': 'embedded', 'source': 'identity--8d87b1fd-d2d6-4c79-84f9-628d395063ea', 'target': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b'}, {'name': 'email address', 'type': 'embedded', 'source': 'identity--d6c46332-e351-41c7-a043-8a9a5eba3705', 'target': 'email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3'}, {'name': 'user account', 'type': 'embedded', 'source': 'identity--d6c46332-e351-41c7-a043-8a9a5eba3705', 'target': 'user-account--83658594-537d-5c32-b9f0-137354bd9bc3'}, {'name': 'from', 'type': 'embedded', 'source': 'email